In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import replicaEVSE.datautils as simdu
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
mode = 'PRIVATE_AUTO'
test = False

#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

df = df[(df['building_type'] != 'mobile') & (df['building_type'] != None)]



###  sample by county: for now use a percent until we have the stock rollover model

`ev_df` should be given to us to run. 

In [2]:
stock_rollover = pd.read_csv(datadir+'LDV_pop_adjusted.csv')
efficiency = pd.read_csv(datadir+'vehicle_inputs.csv')

In [ ]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']
for cartype in personal + commercial:
    for powertrain in  ['EV', 'PHEV']:
        cond = (efficiency['Vehicle_type']==cartype) & ~efficiency['Powertrain'].isin(['ICE-G', 'ICE-D', 'FCEV']) & (efficiency['Powertrain']==powertrain) & (efficiency['Vintage'] == 2022)
        ef = efficiency[cond]['Efficiency'].values[0]
        print(cartype, powertrain, ef)

In [3]:
segment = 'Personal Sedan'
# domicile = 'sfh'
nev_df = stock_rollover[stock_rollover['Powertrain']=='EV'].copy()
# nev_df = nev_df[nev_df['Vehicle_type']==segment].copy()
nev_df = nev_df[nev_df['domicile'] != 'other'].copy()


In [6]:
nev_df.sum()

County          AdamsAdamsAdamsAdamsAdamsAdamsAdamsAdamsAdamsA...
Vehicle_type    Commercial CrossoverCommercial CrossoverCommer...
domicile        mfhsfhmfhsfhmfhsfhmfhsfhmfhsfhmfhsfhmfhsfhmfhs...
Powertrain      EVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEVEV...
2022                                                        88528
2023                                                       235891
2024                                                       432287
2025                                                       616258
2026                                                       813835
2027                                                      1011377
2028                                                      1218660
2029                                                      1393617
2030                                                      1619946
2031                                                      1859807
2032                                                      2116652
2033      

We ingest the number of vehicles in each segment for each year and sample from that subset of the trips table for each county segment and year. This takes > 300 minutes

In [ ]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']

if False:
    for year in np.arange(2022, 2036, 1):
        segment_persons = []
        for segment in personal+commercial:
            seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
            simulation_id = f'{seg_string}_{str(year)}'
            print(simulation_id)
            if "Sedan" in segment:
                eff = 0.25
            elif "Crossover" in segment:
                eff = 0.30
            elif 'Truck' in segment:
                eff = 0.49
            else:
                print('no match')
                eff = 0.3
            print(eff)
            nev_df = stock_rollover[stock_rollover['Powertrain']=='EV'].copy()
            nev_df = nev_df[nev_df['Vehicle_type']==segment].copy()
            nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
            
            # we dont want to include the same people in multiple segments.
            # this results in not enough people in some small counties...
            #if len(segment_persons) == 0:
            #    df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year) 
            #else:
            #    df = df[~df['person_id'].isin(segment_persons)]    
            # ~3 mins
            df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year)
            df_county_subset['efficiency'] = eff
            segment_persons.extend(df_county_subset['person_id'].tolist())
            # ddf = dd.from_pandas(df_county_subset, npartitions=500)
            df_county_subset.to_parquet(os.path.join(datadir, f'county_sample_{simulation_id}.parquet'), 
                                        engine='pyarrow',
                                        compression='snappy')

In [7]:
year = 2030
segment = 'Personal Sedan'
seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
simulation_id = f'{seg_string}_{str(year)}'
df_county_subset = pd.read_parquet(os.path.join(datadir, f'county_sample_{simulation_id}.parquet'))

In [8]:
df_county_subset

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,destination_county,efficiency
10297,1696643382025952548,10234240303577394052,PRIVATE_AUTO,WORK,HOME,0 days 07:20:40,0 days 07:25:17,1.883100,nan,530019505001,46.811524,-119.151463,530019505002,46.811462,-119.182942,residential,single_family,residential,single_family,industrial,industrial,industrial,industrial,46.823401,-119.155263,46.815662,-119.174829,thursday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,WORK,"Adams County, WA",0.25
10298,9032998044962274151,10234240303577394052,PRIVATE_AUTO,SHOP,WORK,0 days 08:14:41,0 days 08:22:48,1.508437,nan,530019505002,46.811462,-119.182942,530019504001,46.828923,-119.159610,industrial,industrial,industrial,industrial,commercial,retail,commercial,retail,46.815662,-119.174829,46.826590,-119.158414,thursday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,PUBLIC,"Adams County, WA",0.25
10299,15263364619448657250,10234240303577394052,PRIVATE_AUTO,WORK,SHOP,0 days 09:12:34,0 days 09:17:44,1.508437,nan,530019504001,46.828923,-119.159610,530019505002,46.811462,-119.182942,commercial,retail,commercial,retail,industrial,industrial,industrial,industrial,46.826590,-119.158414,46.815662,-119.174829,thursday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,WORK,"Adams County, WA",0.25
10300,8036733709150494378,10234240303577394052,PRIVATE_AUTO,WORK,HOME,0 days 09:46:21,0 days 09:50:57,1.883100,nan,530019505001,46.811524,-119.151463,530019505002,46.811462,-119.182942,residential,single_family,residential,single_family,industrial,industrial,industrial,industrial,46.823401,-119.155263,46.815662,-119.174829,saturday,12555405632766289807,530019505001,530019505002,nan,53001950500,53001950500,nan,35_64,54.0,F,white,hispanic_or_latino,20000_40000,25931.0,employed,some_college,not_attending_school,naics423710,spouse,0,nan,driving,owner,same_house,2_person,40000_75000,55103.0,married_couple,2,single_family,core,spanish,46.823401,-119.155263,46.815662,-119.174829,NaN,NaN,worked_in_person,WORK,"Adams County, WA",0.25
10301,8105636708517091878,10234240303577394052,PRIVATE_AUTO,HOME,WORK,0 days 10:15:09,0 days 10:23:55,1.61195